In [2]:
import tabula
import pandas as pd
import numpy as np
import pikepdf
from PyPDF2 import PdfFileReader


In [10]:
#count pages
file = "lista-funduszy-ETF.pdf"
pdf = PdfFileReader(open(file,'rb'))
pages = pdf.getNumPages()

In [11]:
# extract urls from pdf
pdf_file = pikepdf.Pdf.open(file)
urls = []
# iterate over PDF pages
for page in pdf_file.pages:
    for annots in page.get("/Annots"):
        uri = annots.get("/A").get("/URI")
        if uri is not None:
            urls.append(str(uri))
url = pd.DataFrame(urls,columns=['url'])
url['Kod ISIN'] = list(map(lambda x: x.split('/')[-1].split('.')[0],url['url']))
url = url.drop_duplicates(subset='Kod ISIN').reset_index(drop=True)
url

,url,Kod ISIN
0,https://www.mdm.pl/ui-pub/site/oferta_indywidu...,etf_dla_zaawansowanych
1,https://www.mdm.pl/addstatic/kid/DE000A0KRJX4.pdf,DE000A0KRJX4
2,https://www.mdm.pl/addstatic/kid/DE000A1N49P6.pdf,DE000A1N49P6
3,https://www.mdm.pl/addstatic/kid/DE000A2BDEB6.pdf,DE000A2BDEB6
4,https://www.mdm.pl/addstatic/kid/GB00B15KXV33.pdf,GB00B15KXV33
...,...,...
279,https://www.mdm.pl/addstatic/kid/IE00BZ048462.pdf,IE00BZ048462
280,https://www.mdm.pl/addstatic/kid/IE00B44CGS96.pdf,IE00B44CGS96
281,https://www.mdm.pl/addstatic/kid/IE00BZ6V7883.pdf,IE00BZ6V7883
282,https://www.mdm.pl/addstatic/kid/IE00BNH72088.pdf,IE00BNH72088


In [6]:
x = []
for i in range(2,pages):
    df = tabula.read_pdf(file,pages=i+1)
    df = pd.DataFrame(df[0])
    #df.columns = df.iloc[0]
    #df = df.rename(columns={np.nan:'Ticker (KID/KIID - Kliknij)'})
    #df = df[df['Nazwa instrumentu (Nazwa w serwisie)']!='Nazwa instrumentu (Nazwa w serwisie)']
    #df = df.dropna()
    x.append(df)
    print('Next page {}'.format(i))
    

Next page 2
Next page 3
Next page 4
Next page 5
Next page 6
Next page 7
Next page 8
Next page 9
Next page 10
Next page 11
Next page 12
Next page 13


In [7]:
# data frame preparation
df = pd.concat(x)
df.columns = ['Nazwa instrumentu', 'Rynek', 'Waluta', 'Kod ISIN',
       'Ticker (KID/KIID)', 'AuM (MLN)', 'Rep.', 'Opl.Zarz.',
       'Dywidenda']
df = df[(df['Nazwa instrumentu']!='Nazwa instrumentu (Nazwa w serwisie)') & (~df['Nazwa instrumentu'].isnull())]


In [8]:
#https://stackoverflow.com/questions/36007849/how-to-split-rows-in-dataframe-based-on-specific-names-in-python
df1 = df.reset_index(drop=True)
df1['Sector'] = df1.iloc[:,0]
sektor = df1['Nazwa instrumentu'][df1['Rynek'].isnull()].tolist()
mask = df1.Sector.isin(sektor)
mask
df1['Sector'] = df1.Sector[mask]
df1['Sector'] = df1.Sector.fillna(method='ffill')
df1= df1[~df1['Rynek'].isnull()]
df1['Dywidenda'] = list(map(lambda x: ' '.join(x.split()[1:]),df1['Dywidenda'].fillna(df1['Opl.Zarz.'])))
df1['Opl.Zarz.'] = list(map(lambda x: x.split()[0],df1['Opl.Zarz.'].fillna(df1['Dywidenda'])))

In [9]:
result = pd.merge(df1,url,on='Kod ISIN')
result.to_csv('Lista-funduszy-ETF.csv')